In [26]:
import csv #import csv
import numpy as np #for laplace --- np.random.laplace(loc=0.0, scale=1.0, size=None)
import matplotlib.pyplot as plt #plotting
import numpy as np #for laplace --- np.random.laplace(loc=0.0, scale=1.0, size=None)
import pandas as pd #for binning purposes
import random #for synthetic dataset
from random import randrange


def hist_create(inp, epsilon, out, num_queries):
    # inp = input dataset file (csv format)
    # epsilon = privatcy budge ranging from 0 to 1
    # out_data = output synthetic dataset file (csv format)
    # num_queries = number of random queries to run for reporting query error
    # out_error = output relative error
    
    # create histogram based on input dataset file
    
    df = pd.read_csv(inp)
    
    #convert pandas to 3D array
    tD_arr = df.to_numpy()

    #calculate bins

     # determine number of equi-width bins using Freedman-Diaconis rule
    q75, q25 = np.percentile(df, [75,25])
    iqr = q75 - q25

    bin_width = 2 * iqr * len(df)**(-1/3)
    num_bins = round(((df.max() - df.min()) / bin_width).max()) # get the max which is the main attribute


    hist, edges = np.histogramdd(tD_arr, bins = num_bins)

    #apply laplace 
    lp = np.random.laplace(0, 1/epsilon, len(df)) #sensitivity at most 1 and privacy budget user defined epsilon
    #get laplace distribution and get counts
    count, bins = np.histogram(lp, num_bins, density = True)

    #add laplace noise to 3D hist
    df_hist = hist + count

    #generate synthetic dataset
    coord = []

    for i in range(len(df_hist)): # for x variable (which array)
        for j in range(len(df_hist[i])): # for y variable (which row)
            for k in range(len(df_hist[i][j])): # for z variable (which column) also the specific number to iterate based on how many there is for that unit
                if df_hist[i][j][k] > 0:
                    for itr in range(0, int(df_hist[i][j][k])):
                        point = [round(random.uniform(edges[0][i], edges[0][i+1]),0), round(random.uniform(edges[1][j], edges[1][j+1]),0), round(random.uniform(edges[2][k],edges[2][k+1]),0)]
                        coord.append(point)

    #convert synthetic dataset into pandas dataframe for sorting and adding tags
    df_syn = pd.DataFrame(coord)

    col_names = [df.keys()[0], df.keys()[1], df.keys()[2]] #changing column names to match
    df_syn.columns = col_names

    #sort original df and new df similarly
    df_s = df.sort_values(by = [df.keys()[0], df.keys()[1], df.keys()[2]])
    df_syn_s = df_syn.sort_values(by = [df_syn.keys()[0], df_syn.keys()[1], df_syn.keys()[2]])


    #change back to numpy for easy comparison with arithmetic
    true = pd.DataFrame(df_s).to_numpy()
    synth = pd.DataFrame(df_syn).to_numpy()

    #calculate random queries accuracy
    queries = num_queries
    que_result = []


    for i in range(0, queries):
        random_query = randrange(len(df))
        que_result.append(abs(true[random_query] - synth[random_query])/synth[random_query])

    que_error = sum(que_result)/queries

    #calculate relative error
    avg_error = abs((sum(true) - sum(synth)))/ sum(synth)
    
    #write to csv synthetic dataset
    df_syn.to_csv(out)
    
    print('Synthetic dataset is outputted at ', out)
    #manually print relative error and query error
    #queue error
    all_que_error = []
    que_counts = 0
    for i in range(0,len(que_error)):
        if que_error[i] > 0:
            all_que_error.append(que_error[i])
            que_counts = que_counts + 1
        
    print('Queue Error with ', num_queries, ' queues: ', sum(all_que_error)/que_counts)
    
    
    #relative error
    all_avg_error = []
    avg_counts = 0
    for i in range(0,len(avg_error)):
        if avg_error[i] > 0:
            all_avg_error.append(avg_error[i])
            avg_counts = avg_counts + 1
        
    print('Relatie Error: ', sum(all_avg_error)/avg_counts)
    

    
def main():
    
    print("Input CSV Input File in format filename.CSV")
    inp = str(input())
    
    print("Input privacy budget epsilon from 0 to 1")
    epsilon = float(input())
    
    print("Input CSV Output File in format filename.CSV, make sure Output File is in same directory as program")
    out = str(input())
    
    print("Input number of random queries to test for query error out of" + len(pd.read_csv(inp)))
    num_queries = int(input())
    
    hist_create(inp, epsilon, out, num_queries)
    
    

    
    

In [19]:
len(pd.read_csv('adult2.csv'))

32561

In [24]:
len(queries)

NameError: name 'queries' is not defined